In [2]:
from neo4j import GraphDatabase

In [46]:
uri = "neo4j+ssc://f1a01e09.databases.neo4j.io"
username = "neo4j"
password = "xxx"

In [48]:
driver = GraphDatabase.driver(uri,auth=(username,password))

In [49]:
with driver.session() as session:
    pass;

1. Graph Creation:
    
Build a graph database to represent a social network, with nodes for users and relationships for friendships.

Create nodes for users and relationships to model their interactions.

In [21]:
def create_user(tx, username, age):
    tx.run("CREATE (:User {username: $username, age: $age})", username=username, age=age)


with driver.session() as session:
    session.execute_write(create_user, "Julia", 37)
    session.execute_write(create_user, "Simon", 32)
    session.execute_write(create_user, "Monika", 35)
    session.execute_write(create_user, "John", 33)
    session.execute_write(create_user, "Anna", 44)
    session.execute_write(create_user, "Roberts", 56)

In [ ]:
def create_friendship(transaction,name1,name2):
    transaction.run("MATCH (person1:Person {name:$name1}),(person2:Person {name:$name2}) 
                    CREATE (person1)-[:FRIEND_OF]->(person2)",name1=name1,name2=name2)


In [31]:
# Create relationships
def create_friendship(tx, username1, username2):
    tx.run("MATCH (user1:User {username: $username1}), (user2:User {username: $username2}) "
           "CREATE (user1)-[:FRIEND]->(user2)", username1=username1, username2=username2)


with driver.session() as session:
    session.execute_write(create_friendship, "Julia", "Simon")
    session.execute_write(create_friendship, "Julia", "Monika")
    session.execute_write(create_friendship, "Monika", "Simon")
    session.execute_write(create_friendship, "Monika", "John")
    session.execute_write(create_friendship, "John", "Anna")
    session.execute_write(create_friendship, "Anna", "John")

In [30]:
def get_person(tx,username):
    result = tx.run("MATCH (user:User {username:$username}) RETURN user.username,user.age",username=username)
    return [{"username":record["user.username"],"age":record["user.age"]} for record in result]

with driver.session() as session:
    result = session.execute_read(get_person,"Simon")
    print(result)

[{'username': 'Simon', 'age': 32}]


In [41]:
# model user interactions
def create_hobbies(tx, name):
    tx.run("CREATE (h:hobbie {name: $name})", name=name)

with driver.session() as session:
     session.execute_write(create_hobbies, 'cycling')
    session.execute_write(create_hobbies, 'reading')
    session.execute_write(create_hobbies, 'painting')

In [42]:
def users_hobbies(tx, username, name):
    tx.run("MATCH (user:User {username: $username}), (h:hobbie {name: $name})"
            "CREATE (user)-[:LIKES]->(h)", username=username, name=name)
    
with driver.session() as session:
    session.execute_write(users_hobbies, 'Julia', 'cycling')
    session.execute_write(users_hobbies, 'Monika', 'cycling')
    session.execute_write(users_hobbies, 'Simon', 'cycling')
    session.execute_write(users_hobbies, 'Julia', 'cycling')
    session.execute_write(users_hobbies, 'Anna', 'reading')
    session.execute_write(users_hobbies, 'Roberts', 'reading')
    session.execute_write(users_hobbies, 'Anna', 'painting')
    session.execute_write(users_hobbies, 'Monika', 'reading')

2. Querying Graph Data:
    
Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation).

Retrieve common friends between two users in the graph.

In [37]:
# Define the queries
find_friends = """
MATCH (user:User {username: 'Monika'})-[:FRIEND*1..depth]-(friend)
WHERE user <> friend
RETURN DISTINCT friend.username
"""

common_friends = """
MATCH (user1:User {username: 'Julia'})-[:FRIEND]-(commonFriend)
MATCH (user2:User {username: 'Monika'})-[:FRIEND]-(commonFriend)
RETURN DISTINCT commonFriend.username
"""

depth = 2  
with driver.session() as session:
    result1 = session.run(find_friends.replace('depth', str(depth)))
    result2 = session.run(common_friends)

    friends_result = [record["friend.username"] for record in result1]
    common_friends_result = [record["commonFriend.username"] for record in result2]

In [43]:
# Display the results
print("Friends up to depth", depth, ":", friends_result)
print("Common Friends:", common_friends_result)

Friends up to depth 2 : ['John', 'Simon', 'Julia', 'Anna']
Common Friends: ['Simon']


3. Recommendation System:
    
Implement a basic recommendation system by finding users who have similar interests based on their interactions.

Propose movies, books, or other items that a user might be interested in based on their connections' preferences.

In [81]:
def get_similar_user(tx, username):
    result = tx.run(
        "MATCH (user:User {username: $username})-[:LIKES]->(hobbie:Hobbie)<-[:LIKES]-(user2:User) "
        "RETURN user2.username, hobbie.name",
        username=username
    )
    return [{"hobbie": record["hobbie.name"], "similar_user": record["user2.username"]} for record in result]



In [82]:
with driver.session() as session:
    result = session.execute_read(get_similar_user, username="Simon")
    print(result)

[]


In [ ]:
def recommend_item(username):
    query = (
        "MATCH (user:User {username: $username})-[:LIKES]->(hobbie:Hobbie)<-[:LIKES]-(user:User)-[:LIKES]->(hobbie:Hobbie) "
        "WHERE NOT (user)-[:LIKES]->(hobbie) "
        "RETURN DISTINCT hobbie.name AS suggested_item"
    )
    with driver.session() as session:
        result = session.run(query, username=username)
        return [record["suggested_item"] for record in result]


4. Pathfinding:
    
Use graph algorithms in Neo4j to find the shortest path between two nodes in the graph.

Find the path of mutual friends between two users.

In [85]:
def find_shortest_path(tx,username1,username2):
    result = tx.run(
            "match (user1:User {username:$username1}),(user2:User {username:$username2}), path= SHORTESTPATH ((user1)-[*]->(user2))" 
            "RETURN nodes(path),relationships(path)",username1=username1,username2=username2)
    return [{"nodes":record["nodes(path)"],"relationships":record["relationships(path)"]} for record in result]


In [86]:
with driver.session() as session:
    results = session.execute_read(find_shortest_path,"Julia","John")
    nodes = results[0]["nodes"]
    relationships = results[0]["relationships"]

In [88]:
for node in nodes:
    print(node.get("username"))

Julia
Monika
John


In [89]:
for relationship in relationships:
    nodes = relationship.nodes
    print(nodes[0].get("username"),"->",nodes[1].get("username"))

Julia -> Monika
Monika -> John


In [124]:
def find_common_friend(tx, username1, username2):
    query = (
        "MATCH (user1:User {username: $username1})-[:FRIEND]-(commonFriend:User)-[:FRIEND]-(user2:User {username: $username2})"
        "RETURN commonFriend.username AS common_friend"
    )
    result = tx.run(query, username1=username1, username2=username2)
    return result.single()["common_friend"]

In [128]:
with driver.session() as session:
    common_friend = session.execute_read(find_common_friend, 'Julia', 'John')

if common_friend is not None:
    print("Common Friend:", common_friend)
else:
    print("No common friend found.")

Common Friend: Monika


C:\Users\SN\AppData\Local\Programs\Python\Python310\lib\site-packages\neo4j\_sync\work\result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


In [ ]:
driver.close()